# Chapter 4. Basic Quantum Theory

**Programming Drill 4.1.1** Write a program that simulates the first quantum system described in this section. The user should be able to specify how many points the particle can occupy (warning: keep the max number low, or you will fairly quickly run out of memory). The user will also specify a ket state vector by assigning its amplitudes.

The program, when asked the likelihood of finding the particle at a given point, will perform the calculations described in Example 4.1.1. If the user enters two kets, the system will calculate the probability of transitioning from the first ket to the second, after an observation has been made.

In [3]:
def ket_to_posibility(ket: list[complex]):
  norm_squareds = [c.real**2 + c.imag**2 for c in ket]
  total = sum(norm_squareds)
  return [ns/total for ns in norm_squareds]

print("The possibilities that the particle would be at each entry: ", ket_to_posibility([-3-1j, -2j, 1j, 2]))

def transpose_matrix(C: list[list[complex]]):
  res = []
  for i in range(len(C[0])):
    res.append([])
    for j in range(len(C)):
      res[i].append(C[j][i].conjugate())
  return res

def multiply_matrixes(A: list[list[complex]], B: list[list[complex]]):
  ma = len(A)
  na = len(A[0])
  mb = len(B)
  nb = len(B[0])
  if na != mb:
    raise Exception("Inappropriate size!")
  res = []
  for i in range(ma):
    res.append([])
    for j in range(nb):
      sum = 0
      for k in range(na):
        sum += A[i][k]*B[k][j]
      res[i].append(sum)
  return res

def to_column_matrix(row_vector):
  return [[row] for row in row_vector]

import math

def calc_vector_norm(vector: list[int]):
  return math.sqrt(sum(c.real**2 + c.imag**2 for c in vector))

def calc_transition_amplitude(source, target):
  bra = transpose_matrix(to_column_matrix(target))
  ket = to_column_matrix(source)
  denominator = calc_vector_norm(source)*calc_vector_norm(target)
  bracket = multiply_matrixes(bra, ket)[0][0]
  amplitude: complex = bracket/denominator
  posibility = math.sqrt(amplitude.real**2 + amplitude.imag**2)
  return bracket/denominator, round(posibility)

print("The transition amplitude and its corresponding probability: ", calc_transition_amplitude(
  [1, -1j],
  [1j, 1]
))

The possibilities that the particle would be at each entry:  [0.5263157894736842, 0.21052631578947367, 0.05263157894736842, 0.21052631578947367]
The transition amplitude and its corresponding probability:  (-0.9999999999999998j, 1)


**Programming Drill 4.2.1** Continue your simulation of a quantum system by adding observables to the picture: the user will input a square matrix of the appropriate size, and a ket vector. The program will verify that the matrix is hermitian, and if so, it will calculate the mean value and the variance of the observable on the given state.

In [35]:
def is_hermitian(matrix: list[list[complex]]):
  for i in range(len(matrix)):
    for j in range(len(matrix[i])):
      if matrix[i][j] != matrix[j][i].conjugate():
        return False
  return True

print(is_hermitian([
  [-1, -1j],
  [1j, 1]
])) # Example 4.2.1

def apply_observable(ohm, psi):
  result = multiply_matrixes(ohm, to_column_matrix(psi))
  return result
  
print(apply_observable([
  [-1, -1j],
  [1j, 1]
], [-1, -1-1j])) # Example 4.2.1

def observe_mean(ohm, psi):
  target = apply_observable(ohm, psi)
  bra = transpose_matrix(target)
  ket = to_column_matrix(psi)
  mean = multiply_matrixes(bra, ket)[0][0]
  return mean

print(observe_mean([
  [1, -1j],
  [1j, 2]
], [2**0.5/2, 2**0.5/2*1j])) # Example 4.2.5

print(observe_mean([
  [3, 1+2j],
  [1-2j, -1]
], [2**0.5/2, -2**0.5/2])) # Exercise 4.2.9

def observe_variance(ohm, psi):
  mean = observe_mean(ohm, psi)
  mean_identity_matrix = []
  for i in range(len(ohm)):
    mean_identity_matrix.append([])
    for j in range(len(ohm[i])):
      if i == j:
        mean_identity_matrix[i].append(mean)
      else:
        mean_identity_matrix[i].append(0)

  delta_ohm = []
  for i in range(len(ohm)):
    delta_ohm.append([])
    for j in range(len(ohm[i])):
      delta_ohm[i].append(ohm[i][j] - mean_identity_matrix[i][j])
  
  print("> What then is the mean of Δψ(Ω) itself on the normalized state |ψ⟩? A simple calculation shows that it is precisely zero: ", observe_mean(delta_ohm, psi)) # Exercise 4.2.10

  delta_ohm_squared = multiply_matrixes(delta_ohm, delta_ohm)

  return observe_mean(delta_ohm_squared, psi)

print(observe_variance([
  [1, -1j],
  [1j, 2]
], [2**0.5/2, 2**0.5/2*1j])) # Example 4.2.7


True
[[1j], [(-1-2j)]]
(2.5000000000000004+0j)
0j
> What then is the mean of Δψ(Ω) itself on the normalized state |ψ⟩? A simple calculation shows that it is precisely zero:  (-4.163336342344337e-16+0j)
(0.25+0j)


**Programming Drill 4.3.1** Next step in the simulation: when the user enters an observable and a state vector, the program will return the list of eigenvalues of the observable, the mean value of the observable on the state, and the probability that the state will transition to each one of the eigenstates. Optional: plot the corresponding probability distribution.

In [58]:
import numpy as np

def norm(c: complex):
  return (c.real**2 + c.imag**2)**0.5

def measure(observable: list[list[complex]], state: list[complex]):
  eigenvalues, eigenvectors = np.linalg.eig(np.array(observable, dtype=complex))
  print("Eigenvalues of the observable:", eigenvalues)
  mean = observe_mean(observable, state)
  print("Mean value of the observable on the state:", mean)
  probabilities = [norm(calc_transition_amplitude(state, ev)[0])**2 for ev in eigenvectors]
  print("Probability that the state will transition to each one of the eigenstates:", probabilities)

measure([
  [-1, -1j],
  [1j, 1]
], [1/2, 1/2])

measure([
  [-1, -1j],
  [1j, 1]
], [0, 1])

Eigenvalues of the observable: [-1.41421356+0.j  1.41421356+0.j]
Mean value of the observable on the state: 0j
Probability that the state will transition to each one of the eigenstates: [0.4999999999999999, 0.4999999999999999]
Eigenvalues of the observable: [-1.41421356+0.j  1.41421356+0.j]
Mean value of the observable on the state: (1+0j)
Probability that the state will transition to each one of the eigenstates: [0.1464466094067262, 0.8535533905932737]
